In [1]:
#!pip install hyperopt

     ---------------------------------------- 1.6/1.6 MB 11.2 MB/s eta 0:00:00
     ------------------------------------- 200.5/200.5 kB 11.9 MB/s eta 0:00:00


In [2]:
#!python.exe -m pip install --upgrade pip

     ---------------------------------------- 2.1/2.1 MB 10.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2


In [36]:
from sklearn.datasets import make_classification
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
from sklearn import datasets
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error

### HyperOpt 라이브러리의 주요 함수와 개념

- fmin: 이 함수는 최적화 과정을 수행하는 주요 함수입니다. 이 함수는 목표 함수(fn), 검색 공간(space), 최적화 알고리즘(algo), 그리고 최대 평가 횟수(max_evals) 등을 인자로 받습니다. 이 함수는 최적의 하이퍼파라미터 설정을 찾기 위해 주어진 알고리즘을 사용하여 검색 공간을 탐색하고, 각 설정에 대해 목표 함수를 평가합니다.

- tpe: 이것은 Tree-structured Parzen Estimator(TPE) 알고리즘을 나타내며, HyperOpt에서 사용할 수 있는 최적화 알고리즘 중 하나입니다. TPE는 베이지안 최적화 알고리즘의 한 종류로, 이전의 평가 결과를 기반으로 하여 다음에 테스트할 하이퍼파라미터를 선택합니다. 이렇게 하면 더 좋은 결과를 내는 하이퍼파라미터의 영역을 더 자주 탐색하게 됩니다.

- hp: 이것은 HyperOpt의 검색 공간을 정의하는 데 사용되는 함수들을 담고 있는 모듈입니다. 예를 들어, hp.uniform('x', 0, 1)은 0과 1 사이에서 균일하게 분포한 실수 값을 가지는 'x'라는 하이퍼파라미터를 정의합니다. 다른 함수로는 hp.choice (여러 선택 사항 중 하나를 선택), hp.normal (정규 분포에서 값을 선택) 등이 있습니다.

- STATUS_OK: 이것은 목표 함수가 성공적으로 완료되었음을 나타내는 상태 코드입니다. 목표 함수는 손실값과 상태 코드를 포함하는 딕셔너리를 반환해야 합니다. 이 상태 코드는 HyperOpt가 평가를 제대로 수행했는지 판단하는 데 사용됩니다. 만약 다른 값을 반환하면, HyperOpt는 해당 평가가 실패했다고 판단하고, 해당 설정을 더 이상 탐색하지 않습니다.

### Tree-structured Parzen Estimator (TPE)
- 하이퍼파라미터 최적화를 위한 알고리즘 중 하나입니다. Bayesian optimization의 한 종류로, 하이퍼파라미터의 새로운 조합을 선택하는 방법으로 주로 사용됩니다.

- TPE 알고리즘의 핵심 아이디어는 과거의 평가 결과를 기반으로 하이퍼파라미터의 좋은 값과 나쁜 값에 대한 확률 모델을 구축하고, 이 모델을 사용하여 다음에 시도할 하이퍼파라미터 값을 선택하는 것입니다.

- TPE는 이를 위해 두 개의 확률 분포 l(x)와 g(x)를 사용합니다. 여기서 x는 하이퍼파라미터를 의미하며, l(x)는 과거의 좋은 하이퍼파라미터 값들에 대한 분포를 나타내고, g(x)는 모든 과거의 하이퍼파라미터 값들에 대한 분포를 나타냅니다.

- 이 두 분포를 이용해 새로운 하이퍼파라미터 값 x를 선택할 때, l(x)/g(x)가 큰 값, 즉 과거의 좋은 하이퍼파라미터 값들에 더 가까운 값을 선택합니다.

- 이런 방식으로, TPE는 점점 더 좋은 하이퍼파라미터 값을 찾아가는 탐색 과정을 수행합니다. 이는 전통적인 Grid Search나 Random Search에 비해 효율적이며, 보다 적은 평가로 좋은 하이퍼파라미터 값을 찾을 수 있게 합니다.

### make_classification 함수 매개변수

- n_samples: 생성할 샘플의 개수를 지정합니다. 기본값은 100입니다.
- n_features: 독립 변수의 개수를 지정합니다. 기본값은 20입니다.
- n_informative: 종속 변수와 관련된 독립 변수의 수를 지정합니다. 기본값은 2입니다.
- n_redundant: 다른 독립 변수의 선형 조합으로 생성된 독립 변수의 수를 지정합니다. 기본값은 2입니다.
- n_classes: 생성할 클래스(또는 레이블)의 개수를 지정합니다. 기본값은 2입니다.

이 함수는 X와 y의 두 개의 배열을 반환합니다. X는 n_samples x n_features 크기의 2차원 배열로, 독립 변수를 포함하고 있습니다. y는 n_samples 크기의 1차원 배열로, 각 샘플의 클래스 레이블을 포함하고 있습니다.

In [4]:
X, y = make_classification(n_samples = 1000, n_features = 10, n_classes = 2)

In [6]:
def objective(args):
    model = LogisticRegression(C = args["C"])
    
    score = cross_val_score(model, X, y, cv = 5).mean()
    
    loss = 1 - score
    return {"loss" : loss, "status" : STATUS_OK}


space = {"C" : hp.loguniform("C", -5, 0),}

best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 100)

print(best)

100%|████████████████████████████████████████████| 100/100 [00:03<00:00, 30.95trial/s, best loss: 0.050000000000000044]
{'C': 0.1266855107854171}


### HyperOpt 사용법
- 입력 변수명과 입력값의 검색 공간 설정
- 목적 함수의 설정
- 목적 함수의 반환 최솟값을 가지는 최적 입력값을 유추

In [7]:
search_space = {"x" : hp.quniform("x", -10, 10, 1), "y" : hp.quniform("y", -15, 15, 1)}

In [14]:
def objective_func(search_space):
    x = search_space["x"]
    y = search_space["y"]
    retval = x**2 - 20*y
    return retval

trial_val = Trials()

best_01 = fmin(fn = objective_func, space = search_space, algo = tpe.suggest, max_evals = 20
              , trials = trial_val, rstate = np.random.default_rng(seed = 0))

print("best : ", best_01)

100%|███████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 667.04trial/s, best loss: -296.0]
best :  {'x': 2.0, 'y': 15.0}


In [15]:
print(trial_val.results)

[{'loss': -64.0, 'status': 'ok'}, {'loss': -184.0, 'status': 'ok'}, {'loss': 56.0, 'status': 'ok'}, {'loss': -224.0, 'status': 'ok'}, {'loss': 61.0, 'status': 'ok'}, {'loss': -296.0, 'status': 'ok'}, {'loss': -40.0, 'status': 'ok'}, {'loss': 281.0, 'status': 'ok'}, {'loss': 64.0, 'status': 'ok'}, {'loss': 100.0, 'status': 'ok'}, {'loss': 60.0, 'status': 'ok'}, {'loss': -39.0, 'status': 'ok'}, {'loss': 1.0, 'status': 'ok'}, {'loss': -164.0, 'status': 'ok'}, {'loss': 21.0, 'status': 'ok'}, {'loss': -56.0, 'status': 'ok'}, {'loss': 284.0, 'status': 'ok'}, {'loss': 176.0, 'status': 'ok'}, {'loss': -171.0, 'status': 'ok'}, {'loss': 0.0, 'status': 'ok'}]


In [16]:
print(trial_val.vals)

{'x': [-6.0, -4.0, 4.0, -4.0, 9.0, 2.0, 10.0, -9.0, -8.0, -0.0, -0.0, 1.0, 9.0, 6.0, 9.0, 2.0, -2.0, -4.0, 7.0, -0.0], 'y': [5.0, 10.0, -2.0, 12.0, 1.0, 15.0, 7.0, -10.0, 0.0, -5.0, -3.0, 2.0, 4.0, 10.0, 3.0, 3.0, -14.0, -8.0, 11.0, -0.0]}


In [18]:
losses = [loss_dict["loss"] for loss_dict in trial_val.results]

In [20]:
result_df = pd.DataFrame({"x" : trial_val.vals["x"], "y" : trial_val.vals["y"], "losses" : losses})
result_df

,x,y,losses
0,-6.0,5.0,-64.0
1,-4.0,10.0,-184.0
2,4.0,-2.0,56.0
3,-4.0,12.0,-224.0
4,9.0,1.0,61.0
5,2.0,15.0,-296.0
6,10.0,7.0,-40.0
7,-9.0,-10.0,281.0
8,-8.0,0.0,64.0
9,-0.0,-5.0,100.0


### 과제 2_0523
위스콘신 유방암 데이터 세트를 로딩해서 XGBooster 알고리즘으로 모델링 및 평가를 수행하시오. 단, HyperOpt를 이용해서 하이퍼 파라미터를 최적화하시오.

In [22]:
dataset = datasets.load_breast_cancer()
X_features = dataset.data
y_label = dataset.target

df = pd.DataFrame(data = X_features, columns = dataset.feature_names)
df["target"] = y_label
df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


In [68]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y_label,
                                                   test_size = 0.2, random_state = 156)

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 156)

In [27]:
def get_clf_eval(y_test = None, pred = None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = round(accuracy_score(y_test, pred),4)
    precision = round(precision_score(y_test, pred),4)
    recall = round(recall_score(y_test, pred),4)
    f1 = round(f1_score(y_test, pred),4)
    roc_auc = round(roc_auc_score(y_test, pred_proba[:, 1]),4)
    print("오차 행렬")
    print(confusion)
    print(f"정확도 : {accuracy}, 정밀도 : {precision}\n재현율 : {recall}, f1 : {f1}")
    print(f"AUC : {roc_auc}")

In [30]:
xgb_wrapper = XGBClassifier(n_estimators = 400, learning_rate = 0.1, max_depth = 3)
xgb_wrapper.fit(X_train, y_train)
w_preds = xgb_wrapper.predict(X_test)
w_pred_proba = xgb_wrapper.predict_proba(X_test)

In [31]:
get_clf_eval(y_test, w_preds, w_pred_proba)

오차 행렬
[[35  2]
 [ 1 76]]
정확도 : 0.9737, 정밀도 : 0.9744
재현율 : 0.987, f1 : 0.9806
AUC : 0.9951


In [88]:
space = {
    'max_depth': hp.choice('max_depth', range(5, 21)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    "min_child_weight" : hp.quniform("min_child_weight", 1, 2, 1),
    "colsample_bytree" : hp.uniform("colsample_bytree", 0.5, 1),
}

In [79]:
def objective(params):
    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = cross_val_score(model, X_train, y_train, scoring = "accuracy", cv = 5).mean()
    
    loss = 1 - score
    return {"loss" : loss, "status" : STATUS_OK}


In [66]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [89]:
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)


100%|█████████████████████████████████████████████| 100/100 [00:37<00:00,  2.68trial/s, best loss: 0.03076923076923066]


In [90]:
print(best)

{'colsample_bytree': 0.6613498756948235, 'learning_rate': 0.11295919675428781, 'max_depth': 13, 'min_child_weight': 1.0}


In [84]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y_label,
                                                   test_size = 0.2, random_state = 156)

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 156)

In [91]:
xgb_wrapper = XGBClassifier(
    colsmaple_bytree = 0.6613498756948235, learning_rate = 0.11295919675428781, max_depth = 13, min_child_weight = 1)
xgb_wrapper.fit(X_train, y_train)
w_preds = xgb_wrapper.predict(X_test)
w_pred_proba = xgb_wrapper.predict_proba(X_test)

[12:39:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "colsmaple_bytree" } are not used.



In [92]:
get_clf_eval(y_test, w_preds, w_pred_proba)

오차 행렬
[[34  3]
 [ 3 74]]
정확도 : 0.9474, 정밀도 : 0.961
재현율 : 0.961, f1 : 0.961
AUC : 0.9923
